In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Layer
from tensorflow.keras.callbacks import ModelCheckpoint

# tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

### Helper Functions

In [ ]:
# an implication of Pytorch CrossMapLRN2d with Keras
class LRN2D(Layer):
    """
    This code is adapted from pylearn2.
    License at: https://github.com/lisa-lab/pylearn2/blob/master/LICENSE.txt
    """

    def __init__(self, alpha=1e-4, k=2, beta=0.75, n=5):
        if n % 2 == 0:
            raise NotImplementedError("LRN2D only works with odd n. n provided: " + str(n))
        super(LRN2D, self).__init__()
        self.alpha = alpha
        self.k = k
        self.beta = beta
        self.n = n

    def get_output(self, train):
        X = self.get_input(train)
        b, ch, r, c = X.shape
        half_n = self.n // 2
        input_sqr = T.sqr(X)
        extra_channels = T.alloc(0., b, ch + 2*half_n, r, c)
        input_sqr = T.set_subtensor(extra_channels[:, half_n:half_n+ch, :, :], input_sqr)
        scale = self.k
        for i in range(self.n):
            scale += self.alpha * input_sqr[:, i:i+ch, :, :]
        scale = scale ** self.beta
        return X / scale

    def get_config(self):
        return {"name": self.__class__.__name__,
                "alpha": self.alpha,
                "k": self.k,
                "beta": self.beta,
                "n": self.n}

    

# another implication of Pytorch CrossMapLRN2d with Keras
class LocalResponseNormalization(Layer):
  
    def __init__(self, n=5, alpha=1e-4, beta=0.75, k=2, **kwargs):
        self.n = n
        self.alpha = alpha
        self.beta = beta
        self.k = k
        super(LocalResponseNormalization, self).__init__(**kwargs)

    def build(self, input_shape):
        self.shape = input_shape
        super(LocalResponseNormalization, self).build(input_shape)

    def call(self, x):
        _, r, c, f = self.shape 
        squared = K.square(x)
        pooled = K.pool2d(squared, (self.n, self.n), strides=(1,1), padding="same", pool_mode='avg')
        summed = K.sum(pooled, axis=3, keepdims=True)
        averaged = self.alpha * K.repeat_elements(summed, f, axis=3)
        denom = K.pow(self.k + averaged, self.beta)
        return x / denom 
    
    def compute_output_shape(self, input_shape):
        return input_shape

## The Nets

### Eye Net

In [ ]:
eye_input = Input(shape=(224,224,3))
eye_cnn1_out = Conv2D(filters=96,
                       kernel_size=11,
                       strides=4,
                       padding='valid',
                       activation='relu',
                       input_shape=(224,224,3))(eye_input)
eye_max1_out = MaxPool2D(pool_size=3, strides=2)(eye_cnn1_out)
eye_lrn1_out = LRN2D(n=5, alpha=1e-4, beta=0.75, k=1.0)(eye_max1_out)
eye_zro1_out = ZeroPadding2D(padding=2)(eye_lrn1_out)
eye_cnn2_out = Conv2D(filters=256,
                       kernel_size=5,
                       strides=1,
                       padding='valid',
                       #groups=2,
                       activation='relu')(eye_zro1_out)
eye_max2_out = MaxPool2D(pool_size=3, strides=2)(eye_cnn2_out)
eye_lrn2_out = LRN2D(n=5, alpha=1e-4, beta=0.75, k=1.0)(eye_max2_out)
eye_cnn3_out = Conv2D(filters=384,
                       kernel_size=3,
                       strides=1,
                       padding='same',
                       activation='relu')(eye_lrn2_out)
eye_out = Conv2D(filters=64,
                  kernel_size=1,
                  strides=1,
                  padding='valid',
                  activation='relu')(eye_cnn3_out)


### Left Eye Net

In [9]:
eyel_input = Input(shape=(224,224,3))
eyel_cnn1_out = Conv2D(filters=96,
                       kernel_size=11,
                       strides=4,
                       padding='valid',
                       activation='relu',
                       input_shape=(224,224,3))(eyel_input)
eyel_max1_out = MaxPool2D(pool_size=3, strides=2)(eyel_cnn1_out)
eyel_lrn1_out = LRN2D(n=5, alpha=1e-4, beta=0.75, k=1.0)(eyel_max1_out)
eyel_zro1_out = ZeroPadding2D(padding=2)(eyel_lrn1_out)
eyel_cnn2_out = Conv2D(filters=256,
                       kernel_size=5,
                       strides=1,
                       padding='valid',
                       #groups=2,
                       activation='relu')(eyel_zro1_out)
eyel_max2_out = MaxPool2D(pool_size=3, strides=2)(eyel_cnn2_out)
eyel_lrn2_out = LRN2D(n=5, alpha=1e-4, beta=0.75, k=1.0)(eyel_max2_out)
eyel_cnn3_out = Conv2D(filters=384,
                       kernel_size=3,
                       strides=1,
                       padding='same',
                       activation='relu')(eyel_lrn2_out)
eyel_out = Conv2D(filters=64,
                  kernel_size=1,
                  strides=1,
                  padding='valid',
                  activation='relu')(eyel_cnn3_out)
model_eyel = Model(eyel_input, eyel_out)


### Right Eye Net

In [ ]:
eyer_input = Input(shape=(224,224,3))
eyer_cnn1_out = Conv2D(filters=96,
                       kernel_size=11,
                       strides=4,
                       padding='valid',
                       activation='relu',
                       input_shape=(224,224,3))(eyer_input)
eyer_max1_out = MaxPool2D(pool_size=3, strides=2)(eyer_cnn1_out)
eyer_lrn1_out = LRN2D(n=5, alpha=1e-4, beta=0.75, k=1.0)(eyer_max1_out)
eyer_zro1_out = ZeroPadding2D(padding=2)(eyer_lrn1_out)
eyer_cnn2_out = Conv2D(filters=256,
                       kernel_size=5,
                       strides=1,
                       padding='valid',
                       #groups=2,
                       activation='relu')(eyer_zro1_out)
eyer_max2_out = MaxPool2D(pool_size=3, strides=2)(eyer_cnn2_out)
eyer_lrn2_out = LRN2D(n=5, alpha=1e-4, beta=0.75, k=1.0)(eyer_max2_out)
eyer_cnn3_out = Conv2D(filters=384,
                       kernel_size=3,
                       strides=1,
                       padding='same',
                       activation='relu')(eyer_lrn2_out)
eyer_out = Conv2D(filters=64,
                  kernel_size=1,
                  strides=1,
                  padding='valid',
                  activation='relu')(eyer_cnn3_out)
model_eyer = Model(eyer_input, eyer_out)


### Face Net

In [11]:
face_input = Input(shape=(224,224,3))
face_cnn1_out = Conv2D(filters=96,
                       kernel_size=11,
                       strides=4,
                       padding='valid',
                       activation='relu',
                       input_shape=(224,224,3))(face_input)
face_max1_out = MaxPool2D(pool_size=3, strides=2)(face_cnn1_out)
face_lrn1_out = LRN2D(n=5, alpha=1e-4, beta=0.75, k=1.0)(face_max1_out)
face_zro1_out = ZeroPadding2D(padding=2)(face_lrn1_out)
face_cnn2_out = Conv2D(filters=256,
                       kernel_size=5,
                       strides=1,
                       padding='valid',
                       #groups=2,
                       activation='relu')(face_zro1_out)
face_max2_out = MaxPool2D(pool_size=3, strides=2)(face_cnn2_out)
face_lrn2_out = LRN2D(n=5, alpha=1e-4, beta=0.75, k=1.0)(face_max2_out)
face_cnn3_out = Conv2D(filters=384,
                       kernel_size=3,
                       strides=1,
                       padding='same',
                       activation='relu')(face_lrn2_out)
face_cnn4_out = Conv2D(filters=64,
                       kernel_size=1,
                       strides=1,
                       padding='valid',
                       activation='relu')(face_cnn3_out)
face_dns1_out = Dense(128, activation = 'relu')(face_cnn4_out)
face_out = Dense(64, activation = 'relu')(face_dns1_out)
model_face = Model(face_input, face_out)


### Grid Net

In [ ]:
grid_input = Input((shape=(25,25))
grid_flt_out = Flatten()(grid_input)
grid_dns1_out = Dense(128, activation = 'relu')(grid_flt_out)
grid_out = Dense(64, activation = 'relu')(grid_dns1_out)
model_grid = Model(grid_input, grid_out)
                   

## **Merging**

### Final Net

In [ ]:
model = Sequential()


